## Mutation operator based on Oladayo and Mallipeddi - adaptive ensemble ES

In [1]:
import numpy as np
import random

# runs simulation
def simulation(env,x):
    f,p,e,t = env.play(pcont=x)
    return f

def fitness_function(population):
    """calculates fitness function for the population and returns list of fitness function"""
    fitness_values = []
    for member in population:
        fitness = simulation(env,member) #still has to be changed to reflect that it is an average of the enemies
        fitness_values.append(fitness)  # (idk if thats the default, when env includes several enemies)
    return fitness_values


def adaptive_ensemble_mutation(population,p_gaussian,nr_gen,p_mutation,fitness_function):
    """mutation operator that applies gaussian mutation and cauchy distribution adaptively"""
    #initialize variables
    gaussian_rows = int(p_gaussian*population.shape[0])
    gaussian_indx = np.random.choice(population.shape[0],gaussian_rows,replace=False)
    gaussian_pop = population[gaussian_indx]
    cauchy_pop = np.array([individual for individual in population.tolist() if individual not in gaussian_pop.tolist()])
    success_counter=0

    #mutation per operator_subgroup
    for index,g_ind in  enumerate(gaussian_pop):
        for gene in range(len(g_ind)):
            if p_mutation>random.uniform(0,1):
                gaussian_pop[index][gene]+=np.random.normal(0,1) 
    for index,c_ind in enumerate(cauchy_pop):
        for gene in range(len(c_ind)):
            if p_mutation>random.uniform(0,1):                     #stiill have to debug
                cauchy_pop[index][gene]+=np.random.standard_cauchy()

    #form new population and evaluate top percent 
    population = np.concatenate((gaussian_pop,cauchy_pop))
    if nr_gen > 10:
        fitness_pop = fitness_function(population)
        sorted_fitness_ind = sorted(range(len(fitness_pop)), key=lambda x:fitness_pop[x], reverse=True)
        elites = population[sorted_fitness_ind[:int(0.2*population.shape[0])]]
        for element in elites:
            if element in gaussian_pop:
                success_counter+=1
        p_gaussian = success_counter/len(elites)

    return population,p_gaussian

population,p_gaussian = adaptive_ensemble_mutation(population,p_gaussian,nr_gen,p_mutation,fitness_function)

    
